In [35]:
#pip install geopy

In [3]:
import pandas as pd
import os

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [4]:
a_christmas_carol = pd.read_csv("output_files\\a_christmas_carol_gpe.csv")
a_christmas_carol

,COREF,start_token,end_token,prop,cat,text
0,1,2424,2424,PROP,GPE,Bedlam
1,3,3903,3904,PROP,GPE,Camden Town
2,4,4188,4188,PROP,GPE,London
3,5,5132,5132,PROP,GPE,Sheba
4,9,11229,11229,PROP,GPE,Damascus
5,10,11431,11431,PROP,GPE,Halloa
6,26,12735,12735,PROP,GPE,Ebenezer


In [5]:
a_christmas_carol_b = a_christmas_carol.drop(["COREF", "start_token", "end_token", "prop", "cat"], axis = 1)
a_christmas_carol_b["Titles"] = ["A Christmas Carol"] * len(a_christmas_carol)

In [6]:
a_christmas_carol_b

,text,Titles
0,Bedlam,A Christmas Carol
1,Camden Town,A Christmas Carol
2,London,A Christmas Carol
3,Sheba,A Christmas Carol
4,Damascus,A Christmas Carol
5,Halloa,A Christmas Carol
6,Ebenezer,A Christmas Carol


Let's repeat that but automazing it!

In [9]:
# List of input CSV file paths

input_files = [
    r"output_files\a_christmas_carol_gpe.csv",
    r"output_files\a_tale_of_two_cities_gpe.csv",
    r"output_files\barnaby_rudge_gpe.csv",
    r"output_files\bleak_house_gpe.csv", 
    r"output_files\david_copperfield_gpe.csv",
    r"output_files\Dombey_and_son_gpe.csv",
    r"output_files\great_expectations_gpe.csv",
    r"output_files\hard_times_gpe.csv",
    r"output_files\little_dorrit_gpe.csv",
    r"output_files\martin_chuzzlewit_gpe.csv", 
    r"output_files\nicholas_nickleby_gpe.csv",
    r"output_files\old_curiosity_gpe.csv",
    r"output_files\oliver_twist_gpe.csv",
    r"output_files\our_mutual_friend_gpe.csv",
    r"output_files\the_battle_of_life_gpe.csv",
    r"output_files\the_chimes_gpe.csv",
    r"output_files\the_cricket_on_the_heart_gpe.csv",
    r"output_files\the_haunted_man_gpe.csv",
    r"output_files\the_pickwick_papers_gpe.csv"
]

In [10]:
pwd

'C:\\Users\\usuario\\ELENA\\it-training uzh\\it-training uzh\\Python Data Analytics Essentials\\Charles Dickens\\entities\\GPE'

In [11]:
output_dir = "filtered_gpe"
os.makedirs(output_dir, exist_ok=True)

for file_path in input_files:
    df = pd.read_csv(file_path)
    df_b = df.drop(["COREF", "start_token", "end_token", "prop", "cat"], axis=1)
    title = os.path.basename(file_path).replace("_gpe.csv", "")
    df_b["Titles"] = [title] * len(df_b)
    output_path = os.path.join(output_dir, os.path.basename(file_path))
    df_b.to_csv(output_path, index=False)

So now we have all our GPEs saved in the directory "filtered gpe". Now let's combine all of them into a unique csv dataframe

In [12]:
all_dfs = []

for file_path in input_files:
    df = pd.read_csv(file_path)
    df_b = df.drop(["COREF", "start_token", "end_token", "prop", "cat"], axis=1)
    title = os.path.basename(file_path).replace("_gpe.csv", "")
    df_b["Titles"] = [title] * len(df_b)
    all_dfs.append(df_b)

# Concatenate all dataframes into one
combined_df = pd.concat(all_dfs, ignore_index=True)

In [13]:
combined_df

,text,Titles
0,Bedlam,a_christmas_carol
1,Camden Town,a_christmas_carol
2,London,a_christmas_carol
3,Sheba,a_christmas_carol
4,Damascus,a_christmas_carol
...,...,...
1222,America,the_pickwick_papers
1223,Wulter,the_pickwick_papers
1224,Dulwich,the_pickwick_papers
1225,Richmond,the_pickwick_papers


Now let's substitute those titles for ones that look better

In [14]:
# Example mapping dictionary
title_map = {
    "a_christmas_carol": "A Christmas Carol",
    "a_tale_of_two_cities" : "A Tale of Two Cities",
    "barnaby_rudge" : "Barnaby Rudge",
    "bleak_house" : "Bleak House",
    "david_copperfield" : "David Copperfield",
    "Dombey_and_son" : "Dombey and Son",
    "great_expectations" : "Great Expectations",
    "hard_times" : "Hard Times",
    "little_dorrit" : "Little Dorrit",
    "martin_chuzzlewit" : "Martin Chuzzlewit", 
    "nicholas_nickleby" : "Nicholas Nickleby",
    "old_curiosity" : "Old Curiosity",
    "oliver_twist" : "Oliver Twist",
    "our_mutual_friend" : "Our Mutual Friend",
    "the_battle_of_life" : "The Battle of Life",
    "the_chimes" : "The Chimes",
    "the_cricket_on_the_heart" : "The Cricket",
    "the_haunted_man" :  "The Haunted Man",   
    "the_pickwick_papers": "The Pickwick Papers"
}

# Replace the titles using .replace with the mapping
combined_df["Titles"] = combined_df["Titles"].replace(title_map)


In [15]:
combined_df

,text,Titles
0,Bedlam,A Christmas Carol
1,Camden Town,A Christmas Carol
2,London,A Christmas Carol
3,Sheba,A Christmas Carol
4,Damascus,A Christmas Carol
...,...,...
1222,America,The Pickwick Papers
1223,Wulter,The Pickwick Papers
1224,Dulwich,The Pickwick Papers
1225,Richmond,The Pickwick Papers


Ok, now let's go back to one example: "A Christmas Carol"

In [16]:
a_christmas_carol_b

,text,Titles
0,Bedlam,A Christmas Carol
1,Camden Town,A Christmas Carol
2,London,A Christmas Carol
3,Sheba,A Christmas Carol
4,Damascus,A Christmas Carol
5,Halloa,A Christmas Carol
6,Ebenezer,A Christmas Carol


We are going to use the Python Library "Geopy" (https://geopy.readthedocs.io/en/stable/) to add the coordinates (LAT/LON) of the text column.

In [17]:
geolocator = Nominatim(user_agent="my_geo_app")

In [18]:
locations = a_christmas_carol_b["text"].to_list()

In [19]:
# Create empty lists to store results
latitudes = []
longitudes = []

for loc in locations:
    try:
        # Geocode the location name
        location = geolocator.geocode(loc)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
            print(f"{loc}: ({location.latitude}, {location.longitude})")
        else:
            latitudes.append(None)
            longitudes.append(None)
            print(f"{loc}: Location not found")
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)
        print(f"{loc}: Error - {e}")
    
    # Pause to respect usage limits
    time.sleep(1)

# Optionally, add latitudes and longitudes back to a DataFrame column for further use
df = pd.DataFrame({'Location': locations, 'Latitude': latitudes, 'Longitude': longitudes})

Bedlam: (54.0489933, -1.5925865)
Camden Town: (51.5392533, -0.142757)
London: (51.5074456, -0.1277653)
Sheba: (31.2457442, 34.7925181)
Damascus: (39.2884381, -77.2038717)
Halloa: Location not found
Ebenezer: (37.340323, -93.3071343)


In [20]:
df

,Location,Latitude,Longitude
0,Bedlam,54.048993,-1.592586
1,Camden Town,51.539253,-0.142757
2,London,51.507446,-0.127765
3,Sheba,31.245744,34.792518
4,Damascus,39.288438,-77.203872
5,Halloa,NaN,NaN
6,Ebenezer,37.340323,-93.307134


We can see that geopy missinterprets some locations. For example, "Bedlam" refers to a 19th century asylum (https://en.wikipedia.org/wiki/Bethlem_Royal_Hospital), but Geopy has interpret it as a present-time location in the UK (https://www.google.com/search?q=54.048993%09-1.592586&sca_esv=6eed89d831f7228a&source=hp&ei=W64YaYuKAZO0hbIPheXroAo&iflsig=AOw8s4IAAAAAaRi8aylXJmWQ-vbPX--QPS913AFK2RAd&ved=0ahUKEwjLo9HTzfSQAxUTWkEAHYXyGqQQ4dUDCBA&uact=5&oq=54.048993%09-1.592586&gs_lp=Egdnd3Mtd2l6IhM1NC4wNDg5OTMJLTEuNTkyNTg2SNcCUABYAHAAeACQAQCYAZ4BoAGeAaoBAzAuMbgBA8gBAPgBAvgBAZgCAKACAJgDAJIHAKAHLbIHALgHAMIHAMgHAA&sclient=gws-wiz). But it is a minor issue! We can also see that when Geopy does not recognize a location (Halloa) it outputs NaN. 

There is also an error (produced by BookNLP) that taggs "Ebenezer" as a location, when in reality it is "Ebenezer Scrooge" (the main character of the book). So possibly there will be some outliers in our data analysis. To refine it, we should do some close-reading of the outputs.

That being said, results are quite solid!

In [21]:
location = geolocator.geocode("Great Britain")
print((location.latitude, location.longitude))  # Approximate coordinates for London

(54.3151594, -1.9181532)


We can also see that geopy outputs the coordinates of the capital of "Great Britain" (London), if the name of a country is provided. That is great, as we will have many examples of that in our full data analysis. 

Let's now produce a csv dataframe with all the lattitudes and longitudes of our datafarme.

In [22]:
final_locations = combined_df["text"].to_list()

In [23]:
# Create empty lists to store results
latitudes = []
longitudes = []

for loc in final_locations:
    try:
        location = geolocator.geocode(loc)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
        print(f"Processed {loc}: lat={latitudes[-1]}, lon={longitudes[-1]}")
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)
        print(f"Error on {loc}: {e}")
    time.sleep(1)

Processed Bedlam: lat=54.0489933, lon=-1.5925865
Processed Camden Town: lat=51.5392533, lon=-0.142757
Processed London: lat=51.5074456, lon=-0.1277653
Processed Sheba: lat=31.2457442, lon=34.7925181
Processed Damascus: lat=39.2884381, lon=-77.2038717
Processed Halloa: lat=None, lon=None
Processed Ebenezer: lat=37.340323, lon=-93.3071343
Processed England: lat=52.5310214, lon=-1.2649062
Processed France: lat=46.603354, lon=1.8883335
Processed London: lat=51.5074456, lon=-0.1277653
Processed Westminster: lat=51.5004439, lon=-0.1265398
Processed America: lat=39.7837304, lon=-100.445882
Processed Norway: lat=61.1529386, lon=8.7876653
Processed Paris: lat=48.8534951, lon=2.3483915
Processed Newgate: lat=53.189006, lon=-2.8881902
Processed Blackheath: lat=51.470342, lon=0.0058096
Processed Dover: lat=51.1251275, lon=1.3134228
Processed Lorry: lat=48.9892581, lon=6.0849045
Processed Calais: lat=50.9524769, lon=1.8538446
Processed Concord: lat=43.207178, lon=-71.537476
Processed Tellson: lat=-

Processed St. Albans: lat=51.753051, lon=-0.3379675
Processed India: lat=22.3511148, lon=78.6677428
Processed China: lat=35.000074, lon=104.999927
Processed Chesney Wold: lat=52.0231967, lon=-0.756839
Processed England: lat=52.5310214, lon=-1.2649062
Processed Britain: lat=54.3151594, lon=-1.9181532
Processed America: lat=39.7837304, lon=-100.445882
Processed Europe: lat=51.0, lon=10.0
Processed Carboy: lat=43.9402146, lon=0.8416037
Processed Pentonville: lat=51.5321048, lon=-0.1148938
Processed Tooting: lat=51.4278214, lon=-0.1679667
Processed Christendom: lat=52.2596702, lon=-7.0985056
Processed Avignon: lat=43.9492493, lon=4.8059012
Processed Marseilles: lat=41.3308666, lon=-88.7081293
Processed Chesney: lat=6.2558676, lon=-57.3876666
Processed Greece: lat=38.9953683, lon=21.9877132
Processed Chelsea: lat=51.4875167, lon=-0.1687007
Processed ESSE: lat=51.4582235, lon=7.0158171
Processed Plymouth Harbour: lat=50.3683292, lon=-4.1330711
Processed Brighton: lat=50.8214626, lon=-0.14005

Processed Camden Town: lat=51.5392533, lon=-0.142757
Processed Devonshire: lat=50.7241405, lon=-3.6607788
Processed Hanbury: lat=52.2688447, lon=-2.0527598
Processed Piccadilly,--which: lat=None, lon=None
Processed Steerforth: lat=51.4382703, lon=-0.1880335
Processed Omer: lat=31.2659365, lon=34.847616
Processed Norwich: lat=52.6285576, lon=1.2923954
Processed Court: lat=47.2398992, lon=7.337079
Processed Guildford: lat=51.2356068, lon=-0.5732063
Processed Memory: lat=52.4799435, lon=-7.3229516
Processed Sahara: lat=22.25, lon=9.8
Processed Hampstead: lat=51.55653, lon=-0.1783011
Processed Dora: lat=33.7286935, lon=-87.0904188
Processed nankeen: lat=-31.7222903, lon=115.7881665
Processed Peru: lat=-6.8699697, lon=-75.0458515
Processed Putney: lat=51.4625314, lon=-0.2163018
Processed Italy: lat=42.6384261, lon=12.674297
Processed Calcutta: lat=22.5726459, lon=88.3638953
Processed ALL: lat=40.8077797, lon=-84.0572225
Processed Switzerland: lat=46.7985624, lon=8.2319736
Processed Naples: 

Processed Walworth: lat=42.6711513, lon=-88.5467715
Processed Somersetshire: lat=51.1617509, lon=-3.0753867
Processed Soho: lat=51.5140654, lon=-0.1350494
Processed Antwerp: lat=51.2211097, lon=4.3997081
Processed Paris: lat=48.8534951, lon=2.3483915
Processed Kew: lat=51.4771443, lon=-0.2855658
Processed Roscian: lat=-27.4030104, lon=-58.9736472
Processed Denmark: lat=55.670249, lon=10.3333283
Processed Richmond: lat=37.5385087, lon=-77.43428
Processed Richmonds: lat=52.4582766, lon=-1.9545429
Processed Surrey: lat=51.2715316, lon=-0.3414524
Processed Yorkshire: lat=53.9825271, lon=-1.38525
Processed Surrey Richmond: lat=51.4690089, lon=-0.307877
Processed America: lat=39.7837304, lon=-100.445882
Processed Southwark: lat=51.5061349, lon=-0.1011379
Processed Blackfriars: lat=51.5115854, lon=-0.1037671
Processed Waterloo: lat=42.498275, lon=-92.3328743
Processed Westminster: lat=51.5004439, lon=-0.1265398
Processed Vauxhall: lat=51.4874834, lon=-0.1229297
Processed Liverpool: lat=53.407

Processed York: lat=53.9656579, lon=-1.0743052
Processed Durham: lat=54.666667, lon=-1.75
Processed Dunstable: lat=51.8861316, lon=-0.5209341
Processed Strait: lat=65.8455661, lon=-169.3785432
Processed Vanity: lat=51.5148418, lon=-0.1335193
Processed Italy-: lat=42.6384261, lon=12.674297
Processed Italy: lat=42.6384261, lon=12.674297
Processed Jerusalem: lat=31.7788472, lon=35.2257856
Processed Dorsetshire: lat=50.7968369, lon=-2.3447323
Processed Martigny: lat=49.8646862, lon=1.1526967
Processed Geneva: lat=46.2017559, lon=6.1466014
Processed Europe: lat=51.0, lon=10.0
Processed Esquire: lat=51.0785281, lon=-114.1497323
Processed Westminster: lat=51.5004439, lon=-0.1265398
Processed Florence: lat=43.7697955, lon=11.2556404
Processed Brighton: lat=50.8214626, lon=-0.1400561
Processed Mediterranean: lat=35.0000035, lon=19.9999957
Processed Marshongers: lat=None, lon=None
Processed Romely: lat=47.1684237, lon=27.5478274
Processed Mantua: lat=45.1692628, lon=10.6708365
Processed Pisa: la

Processed Parliament: lat=55.6759281, lon=12.5809377
Processed Russia: lat=64.6863136, lon=97.7453061
Processed Cheltenham: lat=51.8995685, lon=-2.0711559
Processed Hammersmith: lat=51.4920377, lon=-0.2236401
Processed the United States: lat=47.8291375, lon=-122.5970742
Processed Syria: lat=34.6401861, lon=39.0494106
Processed Chelsea: lat=51.4875167, lon=-0.1687007
Processed Brompton: lat=54.3617085, lon=-1.4237114
Processed Portsmouth: lat=50.800031, lon=-1.0906023
Processed Guildford: lat=51.2356068, lon=-0.5732063
Processed Southampton: lat=50.9025349, lon=-1.404189
Processed Canterbury: lat=51.2800275, lon=1.0802533
Processed Winchester,--when: lat=38.30314, lon=-77.4596489
Processed Ryde: lat=50.7299865, lon=-1.1603699
Processed Cheapside: lat=51.5138049, lon=-0.0921771
Processed Llandaff: lat=51.495523, lon=-3.2194743
Processed North Wales: lat=52.9487856, lon=-3.7601273
Processed Stratford: lat=52.1927803, lon=-1.70634
Processed Birmingham: lat=52.4796992, lon=-1.9026911
Proces

Processed Chicksey: lat=None, lon=None
Processed Holloway: lat=51.5572453, lon=-0.1183706
Processed Erboff: lat=None, lon=None
Processed Bower: lat=-34.1230917, lon=139.3535605
Processed Arbour: lat=49.2756065, lon=-122.9073653
Processed Rooshan: lat=35.5745334, lon=51.466166
Processed Print: lat=45.4270935, lon=-75.6890919
Processed Chiswick: lat=51.4923137, lon=-0.263818
Processed Clerkenwell: lat=51.5237268, lon=-0.1055555
Processed Agincourt: lat=48.7323472, lon=6.236384
Processed Harmony Jail: lat=25.344146, lon=82.9706047
Processed Shares: lat=9.3749624, lon=-83.7048133
Processed Paris: lat=48.8534951, lon=2.3483915
Processed Parliament: lat=55.6759281, lon=12.5809377
Processed Hymen: lat=-33.9601522, lon=151.0448927
Processed Rhodes: lat=48.7584377, lon=6.8985177
Processed Piccadilly: lat=51.5064454, lon=-0.1434673
Processed Yorkshire: lat=53.9825271, lon=-1.38525
Processed Stucconia: lat=None, lon=None
Processed Wight: lat=50.6710482, lon=-1.3327111
Processed Shanklin: lat=50.6

Processed La: lat=20.0171109, lon=103.378253
Processed Bristol: lat=51.4538022, lon=-2.5972985
Processed Bilson: lat=64.1170886, lon=-21.7821544
Processed Slum: lat=45.4388828, lon=14.0020198
Processed City: lat=-37.8141705, lon=144.9655616
Processed France: lat=46.603354, lon=1.8883335
Processed England--: lat=52.5310214, lon=-1.2649062
Processed Smorltork: lat=None, lon=None
Processed Bury: lat=49.3132772, lon=2.3439338
Processed Cornhill: lat=57.6116818, lon=-2.696266
Processed Scotland: lat=56.7861112, lon=-4.1140518
Processed Tyburn: lat=51.518391, lon=-0.1529642
Processed HIS LONG - LOST PARENT: lat=None, lon=None
Processed Westminster: lat=51.5004439, lon=-0.1265398
Processed Fogg: lat=-1.9139177, lon=11.8962954
Processed Bardell: lat=51.9768712, lon=1.0125251
Processed Ipswich: lat=52.0579324, lon=1.1528095
Processed Chelmsford: lat=51.7345329, lon=0.4730532
Processed Jingle: lat=38.395993, lon=112.085828
Processed Whitechapel: lat=51.5174861, lon=-0.0659685
Processed Newgate: 

In [24]:
print(len(latitudes))
print(len(longitudes))
print(len(final_locations))

1227
1227
1227


In [25]:
# Optionally, add latitudes and longitudes back to a DataFrame column for further use
df = pd.DataFrame({'Location': final_locations, 'Latitude': latitudes, 'Longitude': longitudes})

In [26]:
df

,Location,Latitude,Longitude
0,Bedlam,54.048993,-1.592586
1,Camden Town,51.539253,-0.142757
2,London,51.507446,-0.127765
3,Sheba,31.245744,34.792518
4,Damascus,39.288438,-77.203872
...,...,...,...
1222,America,39.783730,-100.445882
1223,Wulter,NaN,NaN
1224,Dulwich,51.441879,-0.084324
1225,Richmond,37.538509,-77.434280


In [27]:
titles = combined_df["Titles"].to_list()

In [28]:
df["Titles"] = titles

In [29]:
df

,Location,Latitude,Longitude,Titles
0,Bedlam,54.048993,-1.592586,A Christmas Carol
1,Camden Town,51.539253,-0.142757,A Christmas Carol
2,London,51.507446,-0.127765,A Christmas Carol
3,Sheba,31.245744,34.792518,A Christmas Carol
4,Damascus,39.288438,-77.203872,A Christmas Carol
...,...,...,...,...
1222,America,39.783730,-100.445882,The Pickwick Papers
1223,Wulter,NaN,NaN,The Pickwick Papers
1224,Dulwich,51.441879,-0.084324,The Pickwick Papers
1225,Richmond,37.538509,-77.434280,The Pickwick Papers


In [31]:
df.to_csv("dickens_coordinates.csv")